This is a notebook for:
1. Read the cities table
2. Connect to OWM API
3. Collect weather data for cities
4. Save data into DataFrame
5. Update weather table with collected data

The weather DataFrame should contain:
* city_id
* forecast_time
* temp
* feels_like
* wind_speed
* outlook
* pop

### 1. Import of the needed modules, keys and stuff:

In [10]:
import pandas as pd
import requests
from IPython.display import JSON
import json
import keys

In [11]:
openweather_API = keys.openweather_API

In [12]:
# Magic Alchemy stuff:
host = '127.0.0.1'
schema = 'gans'
user = 'root'
password = keys.mysql_PW
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### 2. Setting up a weather DataFrame:

In [13]:
# File created out of cities notebook:
city_df = pd.read_sql('city',con=con)
city_df

,city_id,city,country,population,lat,long
0,1,Budapest,Hungary,1706851,47.4925,19.0514
1,2,Prague,Czech Republic,1275406,50.0833,14.4167


In [14]:
# Empty dict:
weather_dict = {'city_id':[],
                'time':[],
                'temp':[],
                'wind_speed':[]}

In [15]:
# Loop through cities with weather API:
for i, city in enumerate(city_df['city_id']):
    url = f"http://api.openweathermap.org/data/2.5/forecast?lat={city_df.iloc[i]['lat']}&lon={city_df.iloc[i]['long']}&appid={openweather_API}&units=metric"
    response = requests.get(url)
    weather_json = response.json()
    for time in weather_json['list'][:9]:
        weather_dict['city_id'].append(city)
        weather_dict['time'].append(time['dt_txt'])
        weather_dict['temp'].append(time['main']['temp'])
        weather_dict['wind_speed'].append(time['wind']['speed'])

In [16]:
# DataFrame:
weather_df = pd.DataFrame(weather_dict)
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city_id     18 non-null     int64  
 1   time        18 non-null     object 
 2   temp        18 non-null     float64
 3   wind_speed  18 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 704.0+ bytes


In [17]:
# See how it works:
weather_df

,city_id,time,temp,wind_speed
0,1,2023-01-10 15:00:00,5.51,5.73
1,1,2023-01-10 18:00:00,5.71,4.14
2,1,2023-01-10 21:00:00,5.64,4.46
3,1,2023-01-11 00:00:00,4.96,3.82
4,1,2023-01-11 03:00:00,4.25,3.14
5,1,2023-01-11 06:00:00,3.45,2.64
6,1,2023-01-11 09:00:00,5.42,2.47
7,1,2023-01-11 12:00:00,7.66,1.99
8,1,2023-01-11 15:00:00,6.92,0.98
9,2,2023-01-10 15:00:00,7.31,3.57


In [18]:
# Alchemy Magic:
weather_df.to_sql('weather',con=con,if_exists='append',index=False)

18